# Projeto 1 - Prediction of orders

## Bibliotecas

In [46]:
## imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

## Leitura do arquivo

In [47]:
# Leitura do arquivo de entrada
base = pd.read_csv("DMC 2013_task and realclass/transact_train.txt", na_filter=True, na_values='?', sep='|')

In [ ]:
# Info da base
base.describe()

## Pré-processamento

### Mudança de granularidade

In [48]:
# Filtrando da base as últimas sessões
nData = base.groupby('sessionNo').tail(1)
nData.set_index('sessionNo', inplace=True)

In [49]:
# Copiando a base filtrada
dBase = nData.copy()
# dBase = nData.deepcopy()

In [5]:
dBase.describe()

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,bSumPrice,bStep,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
count,50000.000000,50000.000000,50000.000000,50000.000000,49682.000000,49682.000000,49682.000000,50000.000000,49325.000000,49325.000000,49325.000000,25711.000000,27361.000000,27046.000000,27046.000000,27046.000000,27361.000000,27333.000000,27361.000000,27361.000000
mean,14.367060,5.923140,1465.728502,20.233700,72.118160,157.646205,1096.642791,3.207580,86.444819,118.589753,199.754140,3.158804,11980.285223,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675
std,4.638547,0.795063,2314.980566,28.430273,176.181174,295.331281,3083.223860,3.549983,203.739233,232.215900,430.988381,1.809356,7407.139373,2755.134822,131.400171,108.671163,26.743458,12.083048,0.457506,112.447109
min,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,17.000000,1.000000,3.000000
25%,11.000000,5.000000,217.926750,4.000000,7.000000,24.990000,119.960000,1.000000,9.990000,19.990000,34.950000,1.000000,5478.000000,600.000000,480.000000,36.000000,2.000000,36.000000,1.000000,14.000000
50%,15.000000,6.000000,686.784500,10.000000,14.990000,49.990000,358.935000,2.000000,19.990000,39.990000,74.970000,3.000000,11843.000000,1300.000000,520.000000,98.000000,8.000000,45.000000,2.000000,33.000000
75%,18.000000,7.000000,1726.262750,24.000000,39.990000,129.990000,938.637500,4.000000,49.990000,79.990000,199.970000,5.000000,18431.000000,3200.000000,553.000000,215.000000,14.000000,53.000000,2.000000,85.000000
max,23.000000,7.000000,21580.092000,200.000000,5999.990000,6999.990000,115742.000000,108.000000,6999.990000,6999.990000,23116.880000,5.000000,25038.000000,50000.000000,638.000000,600.000000,868.000000,99.000000,3.000000,738.000000


### Novas features

In [50]:
# Lista as colunas numericas
numericas = dBase.columns.difference(['onlineStatus', 'availability', 'customerNo','order'], sort=False)
# Lista as colunas nominais
nominais = dBase.filter(items=['onlineStatus', 'availability']).columns

In [52]:
## Novas variaveis
dBase['bCount/cCount'] = np.where( dBase.cCount == 0, 0, dBase.bCount / dBase.cCount)
# xpto3 = base.groupby('sessionNo')['bSumPrice'].mean()
dBase['End'] = dBase['startHour'] + ((dBase['duration'])/3600)
dBase['cPriceVar'] = dBase['cMaxPrice']-dBase['cMinPrice']
dBase['bPriceVar'] = dBase['bMaxPrice']-dBase['bMinPrice']
dBase['moneyExtra'] = dBase['maxVal']-dBase['bSumPrice']
# xpto4 = base.groupby('sessionNo')['bStep'].median()
# xpto5 = base.groupby('sessionNo')['duration'].mean()

In [32]:
# Merge da colunas
# xpto0 = pd.merge(xpto, xpto2, how='inner', on='sessionNo', left_index=True, sort=False, suffixes=('_max', '_min'), copy=True)

In [33]:
# xpto0

,startHour_max,startWeekday_max,duration_max,cCount_max,cMinPrice_max,cMaxPrice_max,cSumPrice_max,bCount_max,bMinPrice_max,bMaxPrice_max,...,bMaxPrice_min,bSumPrice_min,bStep_min,maxVal_min,customerScore_min,accountLifetime_min,payments_min,age_min,address_min,lastOrder_min
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,59.99,...,59.99,59.99,2.0,600.0,70.0,21.0,1.0,43.0,1.0,49.0
2,6,5,94.469,0,NaN,NaN,NaN,0,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,5,341.613,11,29.99,29.99,109.95,2,29.99,29.99,...,29.99,29.99,4.0,1800.0,475.0,302.0,12.0,45.0,1.0,11.0
4,6,5,42.812,4,4.99,4.99,19.96,1,4.99,4.99,...,4.99,4.99,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,5,2816.046,45,27.85,179.95,1093.72,4,27.85,27.85,...,27.85,27.85,4.0,800.0,503.0,18.0,1.0,46.0,1.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,18,7,7271.812,197,6.99,59.99,4315.03,13,10.00,54.99,...,10.00,10.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49997,18,7,667.219,8,70.99,89.99,667.88,1,89.99,89.99,...,89.99,89.99,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49998,18,7,5988.882,77,59.99,149.99,5253.28,3,59.99,99.99,...,59.99,59.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## Criando novas colunas com a Média
# dBase['media_cCount'] = xpto
# dBase['media_bCount'] = xpto2
# dBase['media_bSumPrice'] = xpto3
# dBase['media_bStep'] = xpto4
# dBase['media_duration'] = xpto5
# dBase['bCount/cCount'] = xpto6

In [38]:
# dBase['bCount/cCount'].describe()

count    50000.000000
mean         0.341954
std          0.309972
min          0.000000
25%          0.120690
50%          0.232143
75%          0.454545
max          1.000000
Name: bCount/cCount, dtype: float64

In [ ]:
dBase.describe()

### Discretizando as features não-númericas

In [7]:
## As colunas 'order', onlineStatus' e 'availability' tem valores não-numericos
## Transformar os valores categoricos em númericos
dBase.loc[dBase['order'] == 'n', 'order'] = 0
dBase.loc[dBase['order'] == 'y', 'order'] = 1
dBase['order']

sessionNo
1        1
2        1
3        1
4        0
5        1
        ..
49996    1
49997    0
49998    0
49999    0
50000    0
Name: order, Length: 50000, dtype: int64

In [8]:
## Substituir os valores y e n por 1 e 0, respectivamente
dBase.loc[dBase['onlineStatus'] == 'n', 'onlineStatus'] = 0
dBase.loc[dBase['onlineStatus'] == 'y', 'onlineStatus'] = 1
# dBase['onlineStatus'][dBase.onlineStatus == 1]
dBase['onlineStatus']

sessionNo
1        1
2        1
3        1
4        1
5        1
        ..
49996    1
49997    1
49998    1
49999    1
50000    1
Name: onlineStatus, Length: 50000, dtype: int64

In [9]:
## Substituir os valores categoricos pelos seus indices do array respectivo
# criando array apenas com os possiveis valores categoricos para a coluna
li = dBase['availability'].dropna().unique().tolist()
li
# substituindo cada categoria pelo index
# dBase['availability'].notnull()
# dBase.loc[pd.notnull(dBase['availability'])]
dBase.loc[dBase['availability'].notnull(), 'availability'] = dBase.loc[dBase['availability'].notnull(), 'availability'].apply(lambda x: li.index(x)).values
dBase.loc[dBase['availability'].notnull()]

,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,bMaxPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
sessionNo,,,,,,,,,,,,,,,,,,,,,
1,6.0,5.0,39.887,1.0,59.99000,59.990000,59.990000,1.0,59.990000,59.990000,...,0,1.0,600.000000,70.000000,21.000000,1.000000,43.000000,1.000000,49.000000,1
2,6.0,5.0,94.469,0.0,72.11816,157.646205,1096.642791,0.0,86.444819,118.589753,...,0,NaN,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675,1
3,6.0,5.0,341.613,11.0,9.99000,29.990000,109.950000,2.0,9.990000,29.990000,...,0,3.0,1800.000000,475.000000,302.000000,12.000000,45.000000,1.000000,11.000000,1
4,6.0,5.0,42.812,4.0,4.99000,4.990000,19.960000,1.0,4.990000,4.990000,...,1,NaN,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675,0
5,6.0,5.0,2816.046,45.0,12.99000,179.950000,1093.720000,4.0,19.990000,27.850000,...,0,4.0,800.000000,503.000000,18.000000,1.000000,46.000000,1.000000,40.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,18.0,7.0,7271.812,197.0,6.99000,59.990000,4315.030000,13.0,9.990000,54.990000,...,0,NaN,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675,1
49997,18.0,7.0,667.219,8.0,70.99000,89.990000,667.880000,1.0,89.990000,89.990000,...,0,NaN,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675,0
49998,18.0,7.0,5988.882,77.0,9.99000,149.990000,5253.280000,3.0,49.950000,99.990000,...,0,NaN,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675,0


In [10]:
# dBase.loc[dBase['availability'].isnull()]
dBase['availability']

sessionNo
1        0
2        0
3        0
4        1
5        0
        ..
49996    0
49997    0
49998    0
49999    0
50000    0
Name: availability, Length: 50000, dtype: int64

### Tratando os valores ausentes

#### Estrategia 1 - Removendo valores ausentes

In [54]:
# Colunas com valores ausentes
dBase.isna().any()
# Total de linhas nulas por coluna
# dBase.isna().sum()
# Removendo todas as linhas (registros) com valores nulos
# dBase.dropna(inplace=True)

startHour       False
startWeekday    False
duration        False
cCount          False
cMinPrice       False
                ...  
payments        False
age             False
address         False
lastOrder       False
order           False
Length: 23, dtype: bool

In [ ]:
dBase.describe()

#### Estratégia 2 - Preencher os valores ausentes com a média e moda

In [5]:
from sklearn.impute import SimpleImputer

In [6]:
# Limpeza dos dados ausentes pela média
imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
imputer = imputer.fit(dBase[numericas])
dBase[numericas] = imputer.transform(dBase[numericas])

# Limpeza dos dados ausentes pela moda para colunas com valores nominais
imputerY = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
imputerY = imputerY.fit(dBase[nominais])
dBase[nominais] = imputerY.transform(dBase[nominais])

# Teste
# writer = pd.ExcelWriter('data/base_limpo.xlsx')
# nData.to_excel(writer,'Salvando progresso')
# writer.save()

#### Estratégia 3 - Preencher os valores ausentes com a máximo e moda

In [53]:
xpto = base.groupby('sessionNo')[numericas].max()

# Limpeza dos dados ausentes pela moda para colunas com valores nominais
imputerY = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
imputerY = imputerY.fit(dBase[nominais])
dBase[nominais] = imputerY.transform(dBase[nominais])

#### Estratégia 4 - Preencher os valores ausentes com a minimo e moda

In [ ]:
xpto2 = base.groupby('sessionNo')[numericas].min()

# Limpeza dos dados ausentes pela moda para colunas com valores nominais
imputerY = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
imputerY = imputerY.fit(dBase[nominais])
dBase[nominais] = imputerY.transform(dBase[nominais])

#### Estratégia 5 - Preencher os valores ausentes com a moda ( mais frequente )

In [ ]:
dBase[numericas] = dBase[numericas].dropna()

# Limpeza dos dados ausentes pela moda para colunas com valores nominais
imputerY = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
imputerY = imputerY.fit(dBase[nominais])
dBase[nominais] = imputerY.transform(dBase[nominais])

#### Estratégia 6 - Preencher os valores ausentes com média e nova categoria

In [45]:
# Limpeza dos dados ausentes pela média
imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
imputer = imputer.fit(dBase[numericas])
dBase[numericas] = imputer.transform(dBase[numericas])

# Limpeza dos dados ausentes pela nova categoria
imputer3 = SimpleImputer(missing_values= np.nan, strategy='constant', fill_value=-1)
imputer3 = imputer3.fit(dBase[nominais])
dBase[nominais] = imputer3.transform(dBase[nominais])

pandas.core.frame.DataFrame

#### Estratégia 7 - Preencher os valores ausentes com máximo e nova categoria

In [ ]:
xpto = base.groupby('sessionNo')[numericas].max()

# Limpeza dos dados ausentes pela nova categoria
imputer3 = SimpleImputer(missing_values= np.nan, strategy='constant', fill_value=-1)
imputer3 = imputer3.fit(dBase[nominais])
dBase[nominais] = imputer3.transform(dBase[nominais])

#### Estratégia 8 - Preencher os valores ausentes com mínimo e nova categoria

In [ ]:
xpto = base.groupby('sessionNo')[numericas].min()

# Limpeza dos dados ausentes pela nova categoria
imputer3 = SimpleImputer(missing_values= np.nan, strategy='constant', fill_value=-1)
imputer3 = imputer3.fit(dBase[nominais])
dBase[nominais] = imputer3.transform(dBase[nominais])

### Dividir em conjunto de treinamento e teste

In [11]:
features = dBase.columns.difference(['customerNo','order'], sort=False)

X = dBase[features].values
y = dBase['order'].values

In [12]:
dBase[features].isna().any()

startHour          False
startWeekday       False
duration           False
cCount             False
cMinPrice          False
cMaxPrice          False
cSumPrice          False
bCount             False
bMinPrice          False
bMaxPrice          False
bSumPrice          False
bStep              False
onlineStatus       False
availability       False
maxVal             False
customerScore      False
accountLifetime    False
payments           False
age                False
address            False
lastOrder          False
dtype: bool

In [13]:
previsores = X.copy()
previsores

array([[6.00000000e+00, 5.00000000e+00, 3.98870000e+01, ...,
        4.30000000e+01, 1.00000000e+00, 4.90000000e+01],
       [6.00000000e+00, 5.00000000e+00, 9.44690000e+01, ...,
        4.49210478e+01, 1.70691861e+00, 7.90076752e+01],
       [6.00000000e+00, 5.00000000e+00, 3.41613000e+02, ...,
        4.50000000e+01, 1.00000000e+00, 1.10000000e+01],
       ...,
       [1.80000000e+01, 7.00000000e+00, 5.98888200e+03, ...,
        4.49210478e+01, 1.70691861e+00, 7.90076752e+01],
       [1.80000000e+01, 7.00000000e+00, 7.15341000e+02, ...,
        2.40000000e+01, 1.00000000e+00, 4.00000000e+00],
       [1.80000000e+01, 7.00000000e+00, 8.29430000e+01, ...,
        5.30000000e+01, 1.00000000e+00, 3.40000000e+01]])

In [137]:
type(previsores)

numpy.ndarray

In [14]:
classe = y.copy()

In [15]:
print(classe)

[1 1 1 ... 0 0 0]


In [ ]:
# imputer = imputer.fit(previsores)
# previsores = imputer.transform(previsores)

In [16]:
pd.isnull(previsores).any()

False

#### Normalização dos dados

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

### Treinamento

In [17]:
# Separando o conjunto de treinamento e teste
# Random Forest Model

X_train, X_test, y_train, y_test = train_test_split(previsores, classe, test_size=0.3)

In [18]:
#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=150, max_features=3)

#Train the model using the training sets
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [19]:
print(clf.feature_importances_)

[0.02813171 0.01027617 0.11328662 0.04329546 0.02751863 0.03040508
 0.0411307  0.04803006 0.02990581 0.03164548 0.0465293  0.34030595
 0.00038579 0.00291337 0.02585175 0.03520051 0.03023514 0.03038198
 0.02455364 0.02463893 0.03537793]


In [20]:
y_pred = clf.predict(X_test)

In [21]:
print("Accuracy: {:.4f}".format(accuracy_score(y_test, y_pred)))
print(confusion_matrix(y_test,y_pred))

Accuracy: 0.8945
[[7139  898]
 [ 684 6279]]


In [ ]:
# A implementação do Cross Validation
from sklearn.model_selection import cross_val_score

#Create a Gaussian Classifier
clf_d=RandomForestClassifier(random_state=0, n_estimators=100)

scores_dt = cross_val_score(clf_d, X_train, y_train, scoring='accuracy', cv=5)

print(scores_dt.mean())